In [1]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.library.parameters import SeasonAll
import pandas as pd 

%load_ext nb_black
%load_ext lab_black


<IPython.core.display.Javascript object>

In [2]:
    player_dict = players.get_players()
kobe = [player for player in player_dict if player['full_name'] == 'Kobe Bryant'][0]
kobe_id = kobe['id'] 

<IPython.core.display.Javascript object>

In [7]:
gamelog_kobe = playergamelog.PlayerGameLog(player_id=kobe_id, season = SeasonAll.all)

In [8]:
df_kobe_games_all = gamelog_kobe.get_data_frames()[0]

In [9]:
df_kobe_games_all['OPPONENT'] = df_kobe_games_all['MATCHUP'].apply(lambda x: x.split()[-1])

In [10]:
df_kobe_games_all.shape

(1346, 28)

In [11]:
# clean up opponent abbreviation
remap = {"UTH": "UTA", "SAN": "SAS", "PHO": "PHX",  "NJN": "BKN", "NOK": "NOP", "NOH":"NOP", "SEA": "OKC", "CHH": "CHA", "VAN": "MEM"}
df_kobe_games_all['OPPONENT'].replace(remap, inplace=True)
df_kobe_games_all['OPPONENT'].value_counts()

LAC    69
GSW    67
MEM    67
DEN    66
OKC    65
MIN    64
PHX    64
SAC    64
POR    62
HOU    61
SAS    61
DAL    60
UTA    60
NOP    41
IND    34
CLE    34
NYK    34
MIA    33
DET    33
TOR    33
PHI    32
MIL    32
WAS    32
CHA    31
BOS    31
CHI    31
ATL    29
ORL    29
BKN    27
Name: OPPONENT, dtype: int64

In [12]:
df_kobe_games_all.columns

Index(['SEASON_ID', 'Player_ID', 'Game_ID', 'GAME_DATE', 'MATCHUP', 'WL',
       'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', 'PLUS_MINUS', 'VIDEO_AVAILABLE', 'OPPONENT'],
      dtype='object')

In [15]:
mean_df = df_kobe_games_all.groupby('OPPONENT').agg(
    **{
    'PTS' : ("PTS", 'mean'),
    'REB' : ("REB", 'mean'),
    'STL' : ("STL", 'mean'),
    'BLK' : ("BLK",'mean'),
    'AST' : ('AST', 'mean'),
    'PLUS MINUS' : ('PLUS_MINUS', 'mean'),
    'Minutes' : ('MIN', 'mean'),
    'FGM' : ('FGM', 'mean'),
    'FG3M' : ('FG3M', 'mean'),
    'FTM' : ('FTM', 'mean'),
    }
    ).round(1)

In [18]:
total_df = df_kobe_games_all.groupby('OPPONENT').agg(
    Minutes =   ('MIN', 'sum'),
    FGM = ('FGM', 'sum'),
    FGA = ('FGA', 'sum'),
    FG3M = ('FG3M', 'sum'),
    FG3A = ('FG3A', 'sum'),
    FTM = ('FTM', 'sum'),
    FTA = ('FTA', 'sum'),  
    ).round(1)

In [11]:
highlight_df = df_kobe_games_all.groupby('OPPONENT').agg(
    Games = ('Game_ID', 'count'),
    Wins = ('WL', lambda x: (x=='W').sum()))

In [12]:
def find_best_game(df, team, col="PTS"):
    sub_df = df[df['OPPONENT']==team]
    max_index = sub_df[col].idxmax()
    max_row = df.iloc[[max_index]]
    return f"{max_row[col].item()} on {max_row['GAME_DATE'].item()}"

In [13]:
highlight_df['Most Points']=highlight_df.apply(lambda x: find_best_game(df_kobe_games_all, x.name), axis=1)
highlight_df

,Games,Wins,Most Points
OPPONENT,,,
ATL,29,18,"41 on NOV 01, 2009"
BKN,27,20,"46 on NOV 27, 2005"
BOS,31,18,"43 on JAN 31, 2007"
CHA,31,19,"58 on DEC 29, 2006"
CHI,31,18,"43 on NOV 20, 2005"
CLE,34,17,"47 on JAN 30, 2001"
DAL,60,42,"62 on DEC 20, 2005"
DEN,66,42,"51 on FEB 12, 2003"
DET,33,22,"40 on NOV 17, 2009"


In [14]:
d = {}
mean_json = mean_df.to_json(orient = "index")
total_json = total_df.to_json(orient = "index")
highlight_json = highlight_df.to_json(orient = "index")
import json
d["mean"] = json.loads(mean_json)
d["total"] = json.loads(total_json)
d["highlight"] = json.loads(highlight_json)
from IPython.display import JSON
JSON(d)

<IPython.core.display.JSON object>

In [15]:
pd.DataFrame(d).to_json("../dist/assets/data/game_stats.json")